Dou et al. (2020) model

Agora em Julia versão 1.5.3

Obs: não consigo aumentar grid para 1000 porque não há memória o suficiente. Vamos uasr grid=100 mesmo

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters

using DataFrames


#packages to increase the speed of the code
using BenchmarkTools
using Profile # @profile command
using ProfileView #profiling using graphs

Game parameters

In [8]:
const μ = Float64(4.566) #number of months in each period
const ρ = Float64(0.884) # (1 - ρ) is the firm's depreciation rate each period
# const ρ = Float64(0.63) #testing new depreciation rate
const β = Float64(9.84) #inverse of the speed of learning
const c0 = Float64(0.044) #fixed cost of going to court
const c1 = Float64(0.0015) #variable cost of going to court #funcionou com esse valor
# const c1 = Float64(0.01)


const hs0 = Float64(0.28) #senior's initial skill level
const hj0 = Float64(0.36) #junior's initial skill level

const λj = Float64(0.346)

#const is to alert Julia that these global variables will not change, so she does not have to worry about then

0.346

Extra game parameters

In [3]:
const λd = Float64(0.5) #chance de D ser chamado a propor ao invés de J ou S

const γ = Float64(0.0) #chance de cram down, sujeito à fatia da torta. Prob(cd) = γ * share

0.0

Counterfactual parameters

In [13]:
const tsym = one(1)
const ac = zero(0.0)
const acs = ac
const acj = ac

0.0

Data

In [6]:
const Vmax = Float64(1.0) #firm's maximum reorganization value
const L = Float64(0.25) #firm's maximum liquidation value
const Ds = Float64(0.32) #senior's debt face value
const Dj = Float64(0.68) #junior's debt face value

const D = Ds + Dj #total debt, used to scale costs

1.0

Initial definitions: costs, maximum value of the firm, maximum number of periods



In [14]:
    #number of periods
    function max_turns(Vmax, L, ρ, c0, c1, ac)
        t = 0

        #using the specification of the theoretical model
        #we are not considering the change in indexation from t to t+1 yet

        #the condition L - c0 - c1*t >= 0 assures that the liquidation payoffs won't be negative

        while( (ρ^(t-1) * Vmax > L) && (L - c0 - c1*t - ac >= 0) )
            t = t+1
        end
        return t

    end


    T = max_turns(Vmax, L, ρ, c0, c1, ac)

13

In [15]:
#checking if T is indeed the final period
@show t = T
@show ρ^(t-1) * Vmax
@show L
@show ρ^(t-1) * Vmax > L 


@show t = T-1
@show ρ^(t-1) * Vmax
@show L
@show ρ^(t-1) * Vmax > L




t = T = 13
ρ ^ (t - 1) * Vmax = 0.2277336360006909
L = 0.25
ρ ^ (t - 1) * Vmax > L = false
t = T - 1 = 12
ρ ^ (t - 1) * Vmax = 0.2576172352948992
L = 0.25
ρ ^ (t - 1) * Vmax > L = true


true

### Important: we will create a game with T+1 periods

So we'll be able to include period t==0 in the arrays as index 1

So the notation is: period t is represented by the index t-1


In [16]:
#cost function
    #cost function
    function Ct(t, c0=c0, c1=c1)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end
#precisa de um 'end' para o if e um 'end' para a função
    
#test
@show Ct(0)    
@show Ct(1)
@show Ct(2)
@show Ct(2) == c0 * D + c1 * (1) * D


Ct(0) = 0
Ct(1) = 0
Ct(2) = 0.0455
Ct(2) == c0 * D + c1 * 1 * D = true


true

In [17]:
#maximum value of reorganization each period
#maximum value of reorganization each period
    function Vt(Vmax, ρ, t, ac)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax - ac
        end


    end

@show Vt(Vmax, ρ, 1, ac)
@show Vt(Vmax, ρ, 2, ac)
@show Vt(Vmax, ρ, 3, ac)
@show Vt(Vmax, ρ, T, ac)
@show Vt(Vmax, ρ, T+1, ac)
#index 1 corresponds to t=0 and 2 to t=1...

Vt(Vmax, ρ, 1, ac) = 1.0
Vt(Vmax, ρ, 2, ac) = 1.0
Vt(Vmax, ρ, 3, ac) = 0.884
Vt(Vmax, ρ, T, ac) = 0.2576172352948992
Vt(Vmax, ρ, T + 1, ac) = 0.2277336360006909


0.2277336360006909

In [18]:
#liquidation payoffs

    function s_L(t, acs, acj)
        return min(L - Ct(t) - acs, Ds)
    end

@show s_L(0, acs, acj)
@show s_L(1, acs, acj)
@show s_L(2, acs, acj)


    function j_L(t, acs, acj)
        return min(L - Ct(t) - s_L(t, acs, acj)- acj, Dj)
    end
    
@show j_L(1, acs, acj)

s_L(0, acs, acj) = 0.25
s_L(1, acs, acj) = 0.25
s_L(2, acs, acj) = 0.20450000000000002
j_L(1, acs, acj) = 0.0


0.0

Skill levels' grid and Probability Mass Function(PMF)

Important: we won't use the skill levels directly in the functions, we will use their indexes. For example, index = 40 corresponds to $\theta_{K,t} = 0.4$.

In [19]:
#grid size
const grid = 100

hlow = 0.01
hhigh = 1.0


hvals = LinRange(hlow, hhigh, grid)

hvals
hvals[6]

0.06

In [20]:
#array to tell the size of the pie at period t, according to reorganization skill levels
#arguments: (period, θkt)
U = zeros(T+1, grid)


for t in 1:T+1
    for h in 1:grid
        U[t,h] = hvals[h] * Vt(Vmax, ρ, t, ac) - Ct(t)
    end
end

In [25]:
#cumulative distribution function
function cdf(x, lt, β=β)
    
    #return the cdf of x given the lower bound lt
    #geq than 1 because of our discretization method 
    if(x > 1.0)
        
        return 1.0
    
    else
        
        if(x >= lt)
            return 1.0 - ((1.0-x)^β)/((1-lt)^β)
        else
            return 0.0
        end
    end
end

#test      
@show cdf(0.5, 1, β)

@show cdf(0.5, 0.4, β)

@show cdf(0.99, 0.01, β)

@show cdf(1, 0.01, β)

@show cdf(1, 0.9, β)
@btime cdf(1, 0.9, β)

cdf(0.5, 1, β) = 0.0
cdf(0.5, 0.4, β) = 0.8337136736331454
cdf(0.99, 0.01, β) = 1.0
cdf(1, 0.01, β) = 1.0
cdf(1, 0.9, β) = 1.0
  1.399 ns (0 allocations: 0 bytes)


1.0

In [26]:
#pmf: each point of the grid is at the center of the cdf
function pmf_cdf(lt, δ = hlow/2, β = β)
    pmf = zeros(grid)
    
    for (i, h) in enumerate(hvals)
        pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
    end
    
    return pmf
end


lt = 0.01
pmf_cdf(lt)

100-element Array{Float64,1}:
 0.048602089527521986
 0.09088953125821697
 0.08301186884522238
 0.07574579901540057
 0.06904947823155516
 0.06288353216214904
 0.05721093367839869
 0.05199688568380789
 0.04720870863310611
 0.04281573260029048
 0.038789193758253626
 0.03510213513518168
 0.03172931151561542
 ⋮
 3.7337122282821156e-10
 1.615731992643532e-10
 6.408462649432067e-11
 2.2833512858255745e-11
 7.107758825952715e-12
 1.856736986383112e-12
 3.83026943495679e-13
 5.651035195342047e-14
 4.9960036108132044e-15
 2.220446049250313e-16
 0.0
 0.0

In [27]:
#generating the pmfs

#probability mass function
pmf = zeros(grid,grid)


for (t, θt) in enumerate(hvals)
    pmf[t,:] = pmf_cdf(θt)
end


#we need a pmf for when we have lkt and want to know θk,t+1
#we will call it 'pmf2'
pmf2 = zeros(grid,grid)

for i in eachindex(hvals)
    for j in eachindex(hvals)
        pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
    end
end

#rows are the skill levels today, columns are the skill levels tomorrow
pmf2[1,:] - pmf[1,:]

#should be nice to plot the two pmfs, so we will get the intuition easily

100-element Array{Float64,1}:
 -0.046239926421080726
 -0.08201062544335427
 -0.06652389304197329
 -0.05297653770265348
 -0.04117778693388258
 -0.03095216055945394
 -0.022138473358402036
 -0.014588889342129194
 -0.008168025651978328
 -0.0027521041142639507
  0.0017718514572064026
  0.0055067740688657385
  0.008546270271650315
  ⋮
  7.710965259700306e-9
  3.4895426000488317e-9
  1.4511143376026725e-9
  5.43807346649936e-10
  1.7875565752175066e-10
  4.95698020399729e-11
  1.093125330283789e-11
  1.7426855209729608e-12
  1.6978822815991093e-13
  7.239768851825965e-15
  5.665150746454088e-17
  8.758153619997799e-22

Arrays with the continuation values

In [29]:
#continuation values
#period t (not t+1), θkt, ℓkt, ℓmt
s_W = zeros(T+1, grid, grid, grid);
j_W = zeros(T+1, grid, grid, grid);

#debtor only needs two slots for the skills
d_W = zeros(T+1, grid, grid);

#optimal payments
#period, θkt, ℓmt, outputs
Pst_array = zeros(T, grid, grid, 6); #we won't need payment proposal for period T, a.k.a. "T+1"
Pjt_array = zeros(T, grid, grid, 6);


#D needs two payment arrays because he will make one offer to each creditor
Pdts = zeros(T, grid, grid, 6);
Pdtj = zeros(T, grid, grid, 6);

In [32]:
#populating the last period with the liquidation values
s_W[T+1, :, :, :] .= s_L(T+1, acs, acj); #we need to use .= because it is a broadcast
j_W[T+1, :, :, :] .= j_L(T+1, acs, acj);

#D has a negative liquidation value to force him never to propose liquidation
d_W[T+1, :,:] .= -one(1.0);

Pkt

In [33]:
#function to select the pmf function as it is writen in the paper

function populate_probm!(t, lmt, probm, pmf, pmf2, grid, tsym)
    #takes the indexes of cutoff and lmt

    #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2
    if(t <= tsym)

        for i in 1:grid
            probm[i] = pmf[lmt, i]
        end

    else

        for i in 1:grid
            probm[i] = pmf2[lmt, i]
        end

    end
end

    
#test
probm = zeros(grid)
@btime populate_probm!(2,50, probm, pmf, pmf2, grid, tsym)

  86.384 ns (0 allocations: 0 bytes)


In [34]:
#cutoff function: upgrade

function getcutoff_manual(t, pkt, hkt, m_W, grid)
    
    
    cmt=Int64(1)
    
    while(m_W[t+1, cmt, cmt, hkt] < pkt)
        cmt+=1
        
        #break if cmt==grid+1
        if(cmt==grid+1)
            return cmt
        end
            
    end

    return cmt
end

#should return 100 because J's continuation value at T is zero
@btime getcutoff_manual(T, 0.01, 50, j_W, grid)


  183.876 ns (0 allocations: 0 bytes)


101

In [35]:
#function to "filter" the cutoff

function cutoff_manual(t, pkt, hkt, lmt, m_W, grid)
    #for now, it will just tell us if the payment offer is negative
    
    #if payment is negative (waiting offer), the cutoff will be 102 (grid+2), a code for the adversary to always reject it
    if(pkt<0)
         return grid+2
    end
    
    cmt = getcutoff_manual(t, pkt, hkt, m_W, grid)
    
    return cmt

    
#     #if there is no cutoff (cutoff==101), then cutoff_m will return 101
#     if(cmt==101)
#         return cmt
#     else
#         return max(cmt, lmt)
#     end
end

@show cutoff_manual(T, -0.01, 50, 40, j_W, grid)
@show cutoff_manual(T, 0.01, 50, 40, j_W, grid)
@btime cutoff_manual(T, 0.01, 50, 40, j_W, grid)

cutoff_manual(T, -0.01, 50, 40, j_W, grid) = 102
cutoff_manual(T, 0.01, 50, 40, j_W, grid) = 101
  186.391 ns (0 allocations: 0 bytes)


101

In [36]:
function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)
    
    #calculates the dot product between k_W and pmf
    
    payoff=0.0
    @inbounds for i in hkt:grid
        payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    end
    return payoff
end

#lkt+1 = hkt, so we input it here

dot_W (generic function with 1 method)

In [37]:
function dot_U(t, hkt, grid, U, pkt, pmf)
    
    #calculates the dot product between U and pmf
    payoff=0.0
    @inbounds for i in hkt:grid
        payoff+= (U[t, i] - pkt) * pmf[hkt, i]
    end
    return payoff
end

dot_U (generic function with 1 method)

In [38]:
function lastindmax(x)
    #findmax, but returns higher index when two elements are equal
   k = 1
   m = x[1]
   @inbounds for i in eachindex(x)
       if x[i]>=m
           k = i
           m = x[i]
       end
   end
   return k
end

lastindmax (generic function with 1 method)

In [40]:
function compute_payoff_pkt2!(t, payoff, pmf, probm, k_W, m_W, U, pkt_grid, hkt, lmt, Vmax, grid)
    
    @inbounds for i in 1:(length(pkt_grid))
        
        pkt = pkt_grid[i]
        cmt = lmt + (i)
        
        
        
        if(pkt<0)
            cmt = grid+2
        end

        if(cmt==grid+2) 
            #means that pkt<0, so it will be always rejected
            #lmt+1 == lmt
            payoff[i] += dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum((probm))
#             function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)
    
#                 #calculates the dot product between k_W and pmf

#                 payoff=0.0
#                 @inbounds for i in hkt:grid
#                     payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
#                 end
#                 return payoff
#             end



        elseif(cmt == grid+1) 
            #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

            #since sum(probm[lmt:grid])==1, we didn't include it here
            payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum((probm))
#             function dot_U(t, hkt, grid, U, pkt, pmf)

#                 #calculates the dot product between U and pmf
#                 payoff=0.0
#                 @inbounds for i in hkt:grid
#                     payoff+= (U[t, i] - pkt) * pmf[hkt, i]
#                 end
#                 return payoff
#             end

        else
            
            #cmt > lmt
                #accepted if hmt+1 <=cmt
            #se a habilidade é igual ao cutoff então o adversário rejeita
                payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum(@view(probm[lmt:(cmt)]))


                #rejected otherwise
                #lmt+1 = cmt
                payoff[i] += dot_W(t, hkt, hkt, cmt, grid, k_W, pmf) * sum(@view(probm[(cmt+1):grid]))
            
        end
    end
end

compute_payoff_pkt2! (generic function with 1 method)

In [42]:
grid_payment = 1000
share = LinRange(hlow, hhigh-hlow, grid_payment)

#testing cutoff in this newer version of Pkt

#function to calculate the optimal payment offer and directly populate the array

function Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax=Vmax, grid=grid, acs=acs, acj=acj, tsym=tsym)

    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]
    probm = zeros(grid)
    populate_probm!(t, lmt, probm, pmf, pmf2, grid, tsym)

    pkt_grid = zeros(size(lmt:grid,1))

    min_share = 0.0001

    #popula o grid de pagamentos de acordo com os valores de continuação do adversário
    for (i, cmt) in enumerate(lmt+1:grid)

        share = m_W[t+1, cmt, cmt, hkt]

        if(U[t,hkt] <= share) #to avoid payment promisses that won't be fulfilled
            pkt_grid[i] = -Vmax

        else
            pkt_grid[i] = share

        end


         if(pkt_grid[i] == zero(1))
                pkt_grid[i] = U[t, hkt] * min_share
            end
    end

    pkt_grid[size(lmt:grid,1)] = m_W[t+1, grid, grid, hkt] + U[t, hkt] * min_share

    payoff = zeros(length(pkt_grid))
    compute_payoff_pkt2!(t, payoff, pmf, probm, k_W, m_W, U, pkt_grid, hkt, lmt, Vmax, grid)

    #payoffs matrix####



    index_reorg = lastindmax(payoff)
    payoff_reorg = payoff[index_reorg]
    payment = pkt_grid[index_reorg] 
    cmt = lmt + (index_reorg) #tem que seguir a mesma regra do cmt de dentro de compute_payoff_pkt!

    if(payment <0)
        cmt = grid+2
    end

#     if(index_reorg==1)

#         payment = -Vmax
#         cmt = grid+2

#     else
#         payment = pkt_grid[index_reorg-1]
#         cmt = lmt + index_reorg - 1
#     end    



    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t, acs, acj)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum(probm)

    #se o valor da torta for negativo, os cálculos de reorganização ficam errados
    #se o valor da torta for negativo, o jogador ou espera ou liquida a firma
    if(U[t,hkt] <= 0.0)
        payoff_reorg = payoff_wait
        cmt = grid+2
        payment = -Vmax
    end

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    if(payoff_max == payoff_wait)
        cmt = grid+2
        payment = -Vmax
    end



    #populates the array
    Pkt_array[t, hkt, lmt, :] .= payment, cmt, payoff_reorg, payoff_wait, payoff_max, policy

end




#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U = T, 100, 60, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U
@show Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax)
@btime Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax)


#testing hkt=97
hkt=97
@show Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax, grid)


#testing hk = 0.5
hkt = 50
@show Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

@btime Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)



Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax) = [1.9561723529489923e-5, 101.0, 0.19559767357136998, 0.18650000000000025, 0.19559767357136998, 3.0]
  2.122 μs (18 allocations: 2.34 KiB)
Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax, grid) = [1.8788871823605225e-5, 101.0, 0.1883011225211499, 0.18650000000000025, 0.1883011225211499, 3.0]
Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U) = [6.680861764744961e-6, 61.0, 0.18003097459183068, 0.18650000000000044, 0.188, 1.0]
  6.560 μs (18 allocations: 2.34 KiB)


6-element view(::Array{Float64,4}, 13, 50, 60, :) with eltype Float64:
  6.680861764744961e-6
 61.0
  0.18003097459183068
  0.18650000000000044
  0.188
  1.0

### Início da função de propor do devedor

In [45]:
function dot_d_W(t, hst, hjt, grid, d_W, pmf)

    #calculates the dot product between k_W and pmf
    #adapted for the debtor

    payoff=0.0
    @inbounds for s in hst:grid
        @inbounds for j in hjt:grid
            payoff += d_W[t+1, s, j] * pmf[hst, s] * pmf[hjt, j]
            
        end
    end
    
    return payoff
end


dot_d_W (generic function with 1 method)

In [46]:
function compute_payoff_pdt!(t, payoff, pmf, d_W, s_W, j_W, U, pst_grid, pjt_grid, hdt, hst, hjt, Vmax, grid)

    @inbounds for s in 1:(length(pst_grid))
        @inbounds for j in 1:(length(pjt_grid))
            
            pst = pst_grid[s]
            cst = hst + (s)
            
            pjt = pjt_grid[j]
            cjt = hjt + (j)
            
            
            if(U[t, hdt] < pst + pjt || pst <0 || pjt < 0)
                
                cst = grid+2
                cjt = grid+2
                
            end
                
            if(cst == grid+2 || cjt == grid+2)
                
                payoff[s,j] +=  dot_d_W(t, hst, hjt, grid, d_W, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, hjt:grid])
            
            elseif(cst == grid+1)
                #S will always accept the payment offer
                
                #accepted if hmt+1 <= cmt for m in {s, j}
                payoff[s,j] += dot_U(t, hdt, grid, U, pst+pjt, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, hjt:cjt])
                
                #case when J rejects and S accepts
                payoff[s,j] += dot_d_W(t, hst, cjt, grid, d_W, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, cjt+1:grid])
                
            elseif(cjt == grid+1)
                #J will always accept the payment offer
                
                #accepted if hmt+1 <= cmt for m in {s, j}
                payoff[s,j] += dot_U(t, hdt, grid, U, pst+pjt, pmf) * sum(pmf[hst, hst:cst]) * sum(pmf[hjt, hjt:grid])
                
                
                #case when J accepts and S rejects
                payoff[s,j] += dot_d_W(t, cst, hjt, grid, d_W, pmf) * sum(pmf[hst, cst+1:grid]) * sum(pmf[hjt, hjt:grid])
                
            else
                
                #accepted if hmt+1 <= cmt for m in {s, j}
                payoff[s,j] += dot_U(t, hdt, grid, U, pst+pjt, pmf) * sum(pmf[hst, hst:cst]) * sum(pmf[hjt, hjt:cjt])
                
                
                #rejected if one of them evolves to a skill higher than the cutoff####
                #we can add a chance of cram down in these two cases
                
                #case when J accepts and S rejects
                payoff[s,j] += dot_d_W(t, cst, hjt, grid, d_W, pmf) * sum(pmf[hst, cst+1:grid]) * sum(pmf[hjt, hjt:cjt])
                
                #case when J rejects and S accepts
                payoff[s,j] += dot_d_W(t, hst, cjt, grid, d_W, pmf) * sum(pmf[hst, hst:cst]) * sum(pmf[hjt, cjt+1:grid])
                
                
                #rejected if both of them have skill levels above their current cutoffs
                payoff[s,j] += dot_d_W(t, cst, cjt, grid, d_W, pmf) * sum(pmf[hst, cst+1:grid]) * sum(pmf[hjt, cjt+1:grid])
                
                
            end
            
        end
        
        
    end
    
end

compute_payoff_pdt! (generic function with 1 method)

In [55]:
function Pdt!(t, hst, hjt, d_W, s_W, j_W, Pdts, Pdtj, pmf, share, U, Vmax=Vmax, grid=grid, acs=acs, acj=acj, tsym=tsym)

    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]


    #the debtor can see the creditor's real skill levels and can use them to reorganize the firm

    hdt = max(hst, hjt)
    pie = U[t, hdt]

    #grid of payment offers to s and to j####

    min_share = 0.0001

    #payment grid to s####
    pst_grid = zeros(size(hst:grid,1))



    #popula o grid de pagamentos de acordo com os valores de continuação do adversário
    for (i, cst) in enumerate(hst+1:grid)

        share = s_W[t+1, cst, cst, hjt]

        if(U[t,hdt] <= share) #to avoid payment promisses that won't be fulfilled
            pst_grid[i] = -Vmax

        else
            pst_grid[i] = share

        end


         if(pst_grid[i] == zero(1))
                pst_grid[i] = U[t, hdt] * min_share
            end
    end



    pst_grid[size(hst:grid,1)] = s_W[t+1, grid, grid, hjt] + U[t, hdt] * min_share

    #payment grid to j####
    pjt_grid = zeros(size(hjt:grid,1))

    #popula o grid de pagamentos de acordo com os valores de continuação do adversário
    for (i, cjt) in enumerate(hjt+1:grid)

        share = j_W[t+1, cjt, cjt, hst]

        if(U[t,hdt] <= share) #to avoid payment promisses that won't be fulfilled
            pjt_grid[i] = -Vmax

        else
            pjt_grid[i] = share

        end


         if(pjt_grid[i] == zero(1))
                pjt_grid[i] = U[t, hdt] * min_share
            end
    end

    pjt_grid[size(hjt:grid,1)] = j_W[t+1, grid, grid, hst] + U[t, hdt] * min_share


    #payoff is a matrix
    #row i and column j indicates the payoff of pst_grid[i] and pjt_grid[j]
    payoff = zeros(length(pst_grid), length(pjt_grid))


    #retomar daqui####

    compute_payoff_pdt!(t, payoff, pmf, d_W, s_W, j_W, U, pst_grid, pjt_grid, hdt, hst, hjt, Vmax, grid)

    #payoffs matrix####


    payoff_reorg, index_reorg = findmax(payoff)

    payment_s = pst_grid[index_reorg[1]] 
    cst = hst + (index_reorg[1]) #tem que seguir a mesma regra do cmt de dentro de compute_payoff_pkt!


    payment_j = pjt_grid[index_reorg[2]]
    cjt = hjt + (index_reorg[2])

    #calculating the optimal policy between liquidating, reorganizing or waiting ####

    #we suppose the debtor receives a negative payoff in liquidation so he never proposes to liquidate
    payoff_liq = -Vmax

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = dot_d_W(t, hst, hjt, grid, d_W, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, hjt:grid])

    #se o valor da torta for negativo, os cálculos de reorganização ficam errados
    #se o valor da torta for negativo, o jogador ou espera ou liquida a firma
    if(U[t,hdt] <= 0.0 || (U[t, hdt] < payment_s + payment_j) || payment_s < 0 || payment_j < 0 )
        payoff_reorg = payoff_wait
        cst = grid+2
        cjt = grid+2
        payment_s = -Vmax
        payment_j = -Vmax
    end

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    if(payoff_max == payoff_wait)
        cst = grid+2
        payment_s = -Vmax

        cjt = grid+2
        payment_j = -Vmax
    end



    #populates the arrays


    Pdts[t, hst, hjt,:] .= payment_s, cst, payoff_reorg, payoff_wait, payoff_max, policy
    Pdtj[t, hst, hjt, :] .= payment_j, cjt, payoff_reorg, payoff_wait, payoff_max, policy

end

Pdt! (generic function with 6 methods)

In [56]:
t = T
hst = 50
hjt = 97
U[t, hjt]

0.18788871823605224

In [57]:
@btime Pdt!(t, hst, hjt, d_W, s_W, j_W, Pdts, Pdtj, pmf, share, U)

  140.200 μs (1442 allocations: 278.00 KiB)


6-element view(::Array{Float64,4}, 13, 50, 97, :) with eltype Float64:
   1.8788871823605225e-5
 100.0
   0.0018011225211496216
  -1.0000000000000013
   0.0018011225211496216
   3.0

Tentando mudar a função Pdt!()

In [ ]:
hdt = max(hst, hjt)
min_share = 0.0001

In [78]:
function compute_pkt_grid(t, pmf, min_share, k_W, U, hdt, hkt, hmt, Vmax, grid)
    
    #hkt is the skill level of creditor k
    #hmt is the skill level of the opponent of k

    size_hkt = size(hkt:grid, 1)

    pkt_grid = zeros(size_hkt, 3)

    # pkt_grid[i, 1] = payment offer
    # pkt_grid[i, 2] = screening cutoff associated with the payment offer
    # pkt_grid[i, 3] = chance that S accepts the payment offer
    # for future works: pkt_grid[i, 4] = payment offer as a share of the pie U[t, hdt]
    # for future works: pkt_grid[i, 5] = chance of cram down, given the share of the pie

    #popula o grid de pagamentos de acordo com os valores de continuação do adversário
    for (i, ckt) in enumerate(hkt+1:grid)

        share = k_W[t+1, ckt, ckt, hmt]

        if(U[t,hdt] <= share) #to avoid payment promisses that won't be fulfilled
            pkt_grid[i, 1] = -Vmax
            pkt_grid[i, 2] = grid+2
            pkt_grid[i, 3] = zero(0.0)
            

        else
            pkt_grid[i, 1] = share
            pkt_grid[i, 2] = ckt
            pkt_grid[i, 3] = sum(pmf[hkt, hkt:ckt])

        end


         if(pkt_grid[i] == zero(1))
                #to avoid offering zero to J at T if J's liquidation payoff is zero at T+1
                pkt_grid[i, 1] = U[t, hdt] * min_share
                pkt_grid[i, 2] = grid+1
                pkt_grid[i, 3] = sum(pmf[hkt, hkt:grid])
            end
    end


    #the last offer is a offer the creditor can't refuse
    pkt_grid[size(hkt:grid, 1), 1] = k_W[t+1, grid, grid, hmt] + U[t, hdt] * min_share
    pkt_grid[size(hkt:grid, 1), 2] = grid+1
    pkt_grid[size(hkt:grid, 1), 3] = sum(pmf[hkt, hkt:grid])


    return pkt_grid
end

compute_pkt_grid (generic function with 1 method)

In [79]:
pst_grid = compute_pkt_grid(t, pmf, min_share, s_W, U, hdt, hst, hjt, Vmax, grid)

51×3 Array{Float64,2}:
 0.1865     51.0  0.258973
 0.1865     52.0  0.396329
 0.1865     53.0  0.510365
 0.1865     54.0  0.604663
 0.1865     55.0  0.682314
 0.1865     56.0  0.745979
 0.1865     57.0  0.797939
 0.1865     58.0  0.840144
 0.1865     59.0  0.874254
 0.1865     60.0  0.901678
 0.1865     61.0  0.923604
 0.1865     62.0  0.941034
 0.1865     63.0  0.954804
 ⋮                
 0.1865     90.0  1.0
 0.1865     91.0  1.0
 0.1865     92.0  1.0
 0.1865     93.0  1.0
 0.1865     94.0  1.0
 0.1865     95.0  1.0
 0.1865     96.0  1.0
 0.1865     97.0  1.0
 0.1865     98.0  1.0
 0.1865     99.0  1.0
 0.1865    100.0  1.0
 0.186519  101.0  1.0

In [82]:
pjt_grid = compute_pkt_grid(t, pmf, min_share, j_W, U, hdt, hjt, hjt, Vmax, grid)

4×3 Array{Float64,2}:
 1.87889e-5  101.0  1.0
 1.87889e-5  101.0  1.0
 1.87889e-5  101.0  1.0
 1.87889e-5  101.0  1.0

In [129]:
function compute_payoff_pdt2(t, pmf, d_W, s_W, j_W, U, pst_grid, pjt_grid, hdt, hst, hjt, Vmax, grid)

    
    size_s = size(pst_grid, 1)
    size_j = size(pjt_grid, 1)
    
    payoff = zeros(size_s, size_j)
    
    
    @inbounds for s in 1:size_s
        pst, cst, probs = pst_grid[s, :]
        cst = Int64(cst)
        @inbounds for j in 1:size_j
            pjt, cjt, probj = pjt_grid[j, :]
            cjt = Int64(cjt)
            
            
            if(U[t, hdt] < pst + pjt || pst <0 || pjt < 0)
                cst = grid+2
                cjt = grid+2
            end
                
            if(cst == grid+2 || cjt == grid+2)
                
                payoff[s,j] +=  dot_d_W(t, hst, hjt, grid, d_W, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, hjt:grid])
            
            elseif(cst == grid+1)
                #S will always accept the payment offer
                
                #accepted if hmt+1 <= cmt for m in {s, j}
                payoff[s,j] += dot_U(t, hdt, grid, U, pst+pjt, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, hjt:cjt])
                
                #case when J rejects and S accepts
                payoff[s,j] += dot_d_W(t, hst, cjt, grid, d_W, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, cjt+1:grid])
                
            elseif(cjt == grid+1)
                #J will always accept the payment offer
                
                #accepted if hmt+1 <= cmt for m in {s, j}
                payoff[s,j] += dot_U(t, hdt, grid, U, pst+pjt, pmf) * probs * sum(pmf[hjt, hjt:grid])
                
                
                #case when J accepts and S rejects
                payoff[s,j] += dot_d_W(t, cst, hjt, grid, d_W, pmf) * (1.0 - probs) * sum(pmf[hjt, hjt:grid])
                
            else
                
                #accepted if hmt+1 <= cmt for m in {s, j}
                payoff[s,j] += dot_U(t, hdt, grid, U, pst+pjt, pmf) * probs * probj
                
                
                #rejected if one of them evolves to a skill higher than the cutoff####
                #we can add a chance of cram down in these two cases
                
                #case when J accepts and S rejects
                payoff[s,j] += dot_d_W(t, cst, hjt, grid, d_W, pmf) * (1.0 - probs) * probj
                
                #case when J rejects and S accepts
                payoff[s,j] += dot_d_W(t, hst, cjt, grid, d_W, pmf) * probs * (1.0 - prob)
                
                
                #rejected if both of them have skill levels above their current cutoffs
                payoff[s,j] += dot_d_W(t, cst, cjt, grid, d_W, pmf) * (1.0 - probs) * (1.0 - prob)
                
                
            end
            
        end
        
        
    end
    
    payoff_reorg, index_reorg = findmax(payoff)
    s = index_reorg[1]
    j = index_reorg[2]
    
    
    payment_s, cst = pst_grid[s, 1:2]
    payment_j, cjt = pjt_grid[j, 1:2]
    
    return payoff_reorg, payment_s, cst, payment_j, cjt
    
end

compute_payoff_pdt2 (generic function with 1 method)

In [130]:
compute_payoff_pdt2(t, pmf, d_W, s_W, j_W, U, pst_grid, pjt_grid, hdt, hst, hjt, Vmax, grid)

(0.0018011225211505097, 0.1865, 99.0, 1.8788871823605225e-5, 101.0)

In [131]:
function Pdt!(t, hst, hjt, d_W, s_W, j_W, Pdts, Pdtj, pmf, share, U, Vmax=Vmax, grid=grid, acs=acs, acj=acj, tsym=tsym)

    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]


    #the debtor can see the creditor's real skill levels and can use them to reorganize the firm

    hdt = max(hst, hjt)
    pie = U[t, hdt]

    min_share = 0.0001

    #retomar daqui####

    payoff_reorg, payment_s, cst, payment_j, cjt = compute_payoff_pdt2(t, pmf, d_W, s_W, j_W, U, pst_grid, pjt_grid, hdt, hst, hjt, Vmax, grid)

    #payoffs matrix####

    #calculating the optimal policy between liquidating, reorganizing or waiting ####

    #we suppose the debtor receives a negative payoff in liquidation so he never proposes to liquidate
    payoff_liq = -Vmax

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = dot_d_W(t, hst, hjt, grid, d_W, pmf) * sum(pmf[hst, hst:grid]) * sum(pmf[hjt, hjt:grid])

    #se o valor da torta for negativo, os cálculos de reorganização ficam errados
    #se o valor da torta for negativo, o jogador ou espera ou liquida a firma
    if(U[t,hdt] <= 0.0 || (U[t, hdt] < payment_s + payment_j) || payment_s < 0 || payment_j < 0 )
        payoff_reorg = payoff_wait
        cst = grid+2
        cjt = grid+2
        payment_s = -Vmax
        payment_j = -Vmax
    end

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    if(payoff_max == payoff_wait)
        cst = grid+2
        payment_s = -Vmax

        cjt = grid+2
        payment_j = -Vmax
    end



    #populates the arrays


    Pdts[t, hst, hjt,:] .= payment_s, cst, payoff_reorg, payoff_wait, payoff_max, policy
    Pdtj[t, hst, hjt, :] .= payment_j, cjt, payoff_reorg, payoff_wait, payoff_max, policy

end

Pdt! (generic function with 6 methods)

In [132]:
@btime Pdt!(t, hst, hjt, d_W, s_W, j_W, Pdts, Pdtj, pmf, share, U)

  53.100 μs (723 allocations: 80.61 KiB)


6-element view(::Array{Float64,4}, 13, 50, 97, :) with eltype Float64:
   1.8788871823605225e-5
 101.0
   0.0018011225211505097
  -1.0000000000000013
   0.0018011225211505097
   3.0

### fim da função de propor do devedor

Fim do debug na função Pkt!

In [134]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T
t = T
for h in 1:grid
    for l in 1:grid
            Pkt!(t, h, l, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U, Vmax, grid) #Pst
            Pkt!(t, h, l, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U, Vmax, grid) #Pjt
            Pdt!(t, h, l, d_W, s_W, j_W, Pdts, Pdtj, pmf, share, U, Vmax, grid)
    end
end

In [135]:
#optimal proposal
function propose(t, hkt, lkt, lmt, Pkt_array)
    
    
    #it just searches for the corresponding Pkt
    payoff_max, policy = @views Pkt_array[t, hkt, lmt, end-1:end]
    
    return payoff_max, policy
end

#test with hk = 0.5 (index 50)
t, hkt, lkt, lmt, Pkt_array = T, 50, 1, 60, Pst_array
@show propose(t, hkt, lkt, lmt, Pkt_array)

#test with hk = 97
hkt = 97
@show propose(t, hkt, lkt, lmt, Pkt_array)


#test with hk = 1.0 (index 100)
hkt = 100
@show propose(t, hkt, lkt, lmt, Pkt_array)

@btime propose(t, hkt, lkt, lmt, Pkt_array)

propose(t, hkt, lkt, lmt, Pkt_array) = (0.188, 1.0)
propose(t, hkt, lkt, lmt, Pkt_array) = (0.1883011225211499, 3.0)
propose(t, hkt, lkt, lmt, Pkt_array) = (0.19559767357136998, 3.0)
  37.097 ns (1 allocation: 32 bytes)


(0.19559767357136998, 3.0)

In [136]:
#"cdf" for the discretized distribution
function cdf_discrete(x, lt, pmf)

    #returns the probability that θt takes a value less than or equal to x, given lt
    #if needed, ie in threshold, we can just subtract 0.01 from x, so cdf_discrete the prob that θt < x, strictly
    return sum(@view(pmf[lt, lt:x]))
end

#test
# @show cdf(0.5, 0.01)

@show cdf_discrete(50, 1, pmf)

@btime cdf_discrete(50,1, pmf)

# @code_warntype cdf_discrete(50,1)

cdf_discrete(50, 1, pmf) = 0.9989088992792265
  92.347 ns (1 allocation: 16 bytes)


0.9989088992792265

In [137]:
#function to calculate the payment offer threshold ϕmt

function threshold_m(t, lkt, lmt, Pmt_array)
    
    #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
    #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
    #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
    ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)
    
    
    #returns error message if threshold does not exist (101 is returned if missing by searchsortedfirst)
#     @assert ts < 101
    
    return ts
    
end

#test 
t, lkt,lmt, Pmt_array = T, 50, 51, Pst_array
@btime threshold_m(t, lkt,lmt,  Pmt_array)




  41.211 ns (0 allocations: 0 bytes)


97

In [138]:
function prob_liq_m(t, ts, lmt, pmf)
    
    #probability that m will propose liquidation next period
    
    #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
    if(t<=1)
        if(lmt >= ts)
            return 0.0
        else
            return 1.0
        end


    else#if t>0(index>1)

        if(lmt >= ts)
            return 0.0
        else
            return cdf_discrete(ts-1,lmt, pmf) #we subtract 1 because we want prob(θt < ts)
        end

    end
    
    
end

t, lkt,lmt, Pmt_array, pmf = T, 50, 51, Pst_array, pmf
ts = threshold_m(t, lkt,lmt,  Pmt_array)
@btime prob_liq_m(t, ts, lmt, pmf)

# @code_warntype prob_liq_m(t, ts, lmt, pmf)
    

  90.283 ns (1 allocation: 16 bytes)


0.9999999999947272

In [139]:
cdf_discrete(100, 99, pmf)

1.0

In [140]:
function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)
    
    #calculates the dot product between k_W and pmf
    #modified to include hkt_next, to use in respond_payment
    
    payoff=0.0
    @inbounds for i in hkt_next:grid
        payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    end
    return payoff
end

dot_W2 (generic function with 1 method)

In [141]:
# function compute_payoff2!(t, payoff, Pkt_grid, cmt, pmf, probm, k_W, m_W, U, hkt, lmt, grid=grid)

function compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid=grid)
    
    
    payoff=zero(0.0)
    
    
    @inbounds for lmt_next in lmt:grid
        pmt = Pmt_array[t, lmt_next, lkt, 1]
        ckt = Int64(Pmt_array[t, lmt_next, lkt, 2])
    
        if(ckt==grid+2)
            #means that the payment offer is negative (waiting offer)
            #the proposal will be rejected for sure, without lowerbound update
#             payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
            payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]
        
        
        elseif(ckt==grid+1)
            #means that there is no cutoff, K will accept the payment offer for all θkt+1
#             payoff += pmt * sum(@view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
            payoff += pmt * sum(@view(pmf[hkt, hkt:grid])) * pmf[lmt, lmt_next] 
            
        else
            
            if(ckt < lkt)
                #the proposal will be rejected for sure, without lowerbound update
#                 payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                
                payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]
                
            
            elseif(ckt==lkt)
                #will accept only if hkt+1==lkt
                payoff += pmt * pmf[hkt, lkt] * pmf[lmt,lmt_next]
                
                #rejects otherwise
#                 payoff += dot(@view(k_W[t+1, lkt+1:end, lkt, lmt_next]), @view(pmf[hkt, lkt+1:end])) * pmf[lmt, lmt_next]
                
                payoff+= dot_W2(t, hkt, lkt+1, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]
                
#                 function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)
    
#                     #calculates the dot product between k_W and pmf
#                     #modified to include hkt_next, to use in respond_payment

#                     payoff=0.0
#                     @inbounds for i in hkt_next:grid
#                         payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
#                     end
#                     return payoff
#                 end
                
                
            else
            #now ckt > lkt
                
                if(ckt < hkt)
                    #he rejects, but there is an update in the lowerbound lkt+1==ckt
#                     payoff += dot(@view(k_W[t+1, hkt:end, ckt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                
                    payoff += dot_W2(t, hkt, hkt, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]
                    
                else
                    
                    #receives pmt if his skill level is below or equal to the cutoff
#                     payoff += pmt * sum(@view(pmf[hkt, hkt:ckt])) * pmf[lmt, lmt_next]
                    
                    payoff += pmt * sum(@view(pmf[hkt, hkt:(ckt)])) * pmf[lmt, lmt_next]
                    
                    
                    #receives continuation value k_W if his skill level greather than the cutoff
#                     payoff += dot(@view(k_W[t+1, ckt+1:end , ckt, lmt_next]), @view(pmf[hkt, ckt+1:end])) * pmf[lmt, lmt_next]
                    
                    
#                     payoff+= dot_W2(t, hkt, ckt+1, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]
                    
                    payoff+= dot_W2(t, hkt, ckt+1, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]
                    
                                        
#                     function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

#                         #calculates the dot product between k_W and pmf
#                         #modified to include hkt_next, to use in respond_payment

#                         payoff=0.0
#                         @inbounds for i in hkt_next:grid
#                             payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
#                         end
#                         return payoff
#                     end

                    
                    
                end
            end
        end
    end

    
    return payoff
end

    

compute_payoff_pmt (generic function with 2 methods)

In [142]:
#expected payoff of answering a payment proposal

function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf, grid=grid)
    
    
    #the minimum level of hmt necessary for m to offer a payment
    #it will be used as hmt
    ts = threshold_m(t, lkt, lmt, Pmt_array)
    
    #updated lower bound
    lmt = max(ts, lmt)
    
    return compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid)

end
        
        

#test1: expected payoff of S
t, hkt, lkt, lmt, Pmt_array, k_W, pmf = T, 50, 50, 50, Pjt_array, s_W, pmf
@btime respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf, grid)


#o tempo maior dessa função é porque ela não faz o update lmt = max(ts, lmt)
#a ideia é só checar a quantidade de alocações
@btime compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid)

  279.730 ns (1 allocation: 16 bytes)
  2.411 μs (1 allocation: 16 bytes)


0.18650000000000036

In [143]:
# quase mesma função que foi criada para acelerar Pkt!(), mas aqui não subtrai pkt
function dot_U2(t, hkt, grid, U, pmf)
    
    #calculates the dot product between U and pmf
    payoff=0.0
    @inbounds for i in hkt:grid
        payoff+= U[t, i] * pmf[hkt, i]
    end
    return payoff
end

dot_U2 (generic function with 1 method)

In [144]:
#expected payoff of answering a liquidation proposal

function respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, grid=grid)
    
    #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
    #U_{t} (\theta_{t+1}) because the skill level is of the next period
    #dot is matrix multiplication
    payoff_reorg = dot_U2(t, hkt, grid, U, pmf) - m_L(t)
    
    payoff_liq = k_L(t)
    
    return max(payoff_liq, payoff_reorg)
end


#what happens if J proposes to liquidation at the penultimate period and S answers it?
#hkt = 0.5
t, hkt, lkt, lmt, k_L, m_L, pmf, U = T, 50, 50, 50, s_L, j_L, pmf, U
@show respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
@show dot(pmf[hkt, hkt:end], (U[t, hkt:end] .- m_L(t)))
@show k_L(t)

@btime respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)

#changing just hkt
#hkt = 1.0
hkt = 100
@show respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
@show dot(pmf[hkt, hkt:end], (U[t, hkt:end] .- m_L(t)))
@show k_L(t)


@btime respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)

LoadError: [91mMethodError: no method matching j_L(::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  j_L(::Any, [91m::Any[39m, [91m::Any[39m) at In[18]:12[39m

In [145]:
# @code_warntype respond_liq(t, hkt, lkt, lmt, k_L, m_L)

In [146]:
#expected payoff of being called to respond
function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U, grid=grid)
    
    probm_liq = prob_liq_m(t, threshold_m(t, lkt, lmt, Pmt_array), lmt, pmf)
    
    
    return (1.0 - probm_liq) * respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf) + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, grid)
end

#test
t, hkt, lkt, lmt, k_L, m_L, Pmt_array, k_W, pmf, U = T, 50, 50, 50, s_L, j_L, Pst_array, j_W, pmf, U


@btime respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U)

LoadError: [91mMethodError: no method matching j_L(::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  j_L(::Any, [91m::Any[39m, [91m::Any[39m) at In[18]:12[39m

### Populating the game's arrays

#### 1,2,3 testing

In [ ]:
@time begin
    
    t = T
    
    for h in 1:grid
        for l in 1:grid
            Pkt!(t, h, l, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U, Vmax, grid) #Pst
            Pkt!(t, h, l, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U, Vmax, grid) #Pjt
        end
    end
    
end

In [ ]:
#testing functions
# t, hkt, lkt, lmt = T-1, 0.6, 0.4, 0.5
t, hkt, lkt, lmt = T, 60, 40, 50

In [ ]:
#s_propose
@btime propose(t, hkt, lkt, lmt, Pst_array)

In [ ]:
#threshold_j
@btime threshold_m(t, lkt, lmt, Pjt_array)

In [ ]:
#threshold_s
#note that threshold_j < threshold_s because j_L(t) < s_L(t) in this case
@btime threshold_m(t, lkt, lmt, Pst_array)

In [ ]:
#s_respond
@btime respond(t, hkt, lkt, lmt, Pjt_array, s_W, s_L, j_L, pmf, U)

In [ ]:
#this last one is especially important since it will be repeated a million times each period
@btime λj * respond(t, hkt, lkt, lmt, Pjt_array, s_W, s_L, j_L, pmf, U) + (1-λj) * propose(t, hkt, lkt, lmt, Pst_array)[1]

#### Populating the penultimate period of the game

Indexed by T

In [ ]:
function populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)
    
    @inbounds for l in 1:grid
        for h in 1:grid
            Pkt!(t, h, l, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U, Vmax, grid) #Pst
            Pkt!(t, h, l, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U, Vmax, grid) #Pjt
        end
    end

end

In [ ]:
@btime populate_pkt_manual!(T, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

In [ ]:
function populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)
    
    for lm in 1:grid
        for lk in 1:grid
           for hk in lk:grid #populates only if hk >= lk, saving time
                s_W[t, hk, lk, lm] = λj * respond(t, hk, lk, lm, Pjt_array, s_W, s_L, j_L, pmf, U) + (1-λj) * propose(t, hk, lk, lm, Pst_array)[1]
                j_W[t, hk, lk, lm] = λj * propose(t, hk, lk, lm, Pjt_array)[1] + (1-λj) * respond(t, hk, lk, lm, Pst_array, j_W, j_L, s_L, pmf, U)
                
            end
        end
    end
end


In [ ]:
@btime populate_wkt_manual!(T, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)
#não usei λj como argumento porque ele aumenta a quantidade de alocações...

In [ ]:
function populate_periods_manual!(tfinal, tbegin, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)
    
    @time begin


        for t in tfinal:-1:tbegin

            @time begin
                #populating Pkt
                populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

                #populating Wkt
                populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)
            end

        end
        
    end
        
end

### Populating the whole game

In [ ]:
#usando a versão manual
populate_periods_manual!(T, 1, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

Time necessary to populate the whole game

08/01/21: 32 minutes

12/01/21:  6.89 minutes (version 5 of respond_payment)

13/01/21: 4.081 minutes (version 5 of respond_payment (fixed) and optimized version of Pkt)

26/01/21: 5 minutes and 40 seconds (grid_payment=1000)

26/01/21: 4min15s (loop populating only if hk>=lk)

28/03/21, 19h50: 138s; 281.21M allocations: 55.884GiB, 6.17% gc time

28/03/21, 20h10: 130.811769 seconds (254.95 M allocations: 55.297 GiB, 6.55% gc time)

28/03/21, 21h07: 133.401700 seconds (254.95 M allocations: 55.297 GiB, 6.92% gc time)

29/03/21, 08h46: 113.804470 seconds (161.72 M allocations: 27.636 GiB, 3.34% gc time)

29/03/21, 10h17: 87.952999 seconds (135.46 M allocations: 17.426 GiB, 2.26% gc time)

30/03/21, 10h54:  84.616759 seconds (122.33 M allocations: 12.974 GiB, 1.82% gc time)

30/03/21, 19h18:  78.780158 seconds (109.20 M allocations: 7.869 GiB, 0.87% gc time)

30/03/21, 20h27:  84.408751 seconds (108.99 M allocations: 3.908 GiB, 0.76% gc time)

#### Tempo necessário para popular período zero (t==1)

In [ ]:
@btime populate_pkt_manual!(1, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

In [ ]:
@btime populate_wkt_manual!(1, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)

### Graphs

In [ ]:
#definindo as funções de S e J só para facilitar nos gráficos

function s_propose(t, hk, lk, lm)
    return propose(t, hk, lk, lm, Pst_array)
end

function j_propose(t, hk, lk, lm)
    return propose(t, hk, lk, lm, Pjt_array)
end

function s_respond(t, hk, lk, lm)
    return respond(t, hk, lk, lm, Pjt_array, s_W, s_L, j_L, pmf, U)
end

function j_respond(t, hk, lk, lm)
    return respond(t, hk, lk, lm, Pst_array, j_W, j_L, s_L, pmf, U)
end


In [ ]:
using Gadfly

In [ ]:
function graf1(t, lkt1, lkt2, lmt1, lmt2, f)
    
    set_default_plot_size(16cm, 10cm)

    #let's cut the X axis to show only values where hkt >=  max(lkt1, lkt2)
    cut = max(lkt1,lkt2)
    X = hvals[cut:grid]

    #arrays to store the outputs
    y1 = zeros(length(hvals[cut:end]))
    y2 = zeros(length(hvals[cut:end]))


    #by default, yi will get lkti and lmti

    #condition to avoid getting the wrong value when the function returns a tuple (ie, k_propose)
    if(f(t, 1, lkt1, lmt1)==Float64)

        for (i, hkt) in enumerate(cut:grid)
            y1[i] = f(t, hkt, lkt1, lmt1)
        end

        for (i, hkt) in enumerate(cut:grid)
            y2[i] = f(t, hkt, lkt2, lmt2)
        end
    else
        for (i, hkt) in enumerate(cut:grid)
            y1[i] = f(t, hkt, lkt1, lmt1)[1]
        end

        for (i, hkt) in enumerate(cut:grid)
            y2[i] = f(t, hkt, lkt2, lmt2)[1]
        end
    end

    
    #plotting
    
    color_y1 = "#1f77b4"
    color_y2 = "#ff7f0e"
    
    
#     possible line_styles=[:solid :dash :dot :dashdot :dashdotdot]
    p = plot()
    
    push!(p, layer(x=X, y=y1, Geom.line,
            Theme(default_color=color(color_y1),
                line_style=[:dashdotdot],
                line_width=0.4mm)))
    
    
    push!(p, layer(x=X, y=y2, Geom.line, 
            Theme(default_color=color(color_y2),
                line_style=[:dash],
                line_width=0.4mm)))
    
    


    push!(p, Guide.manual_color_key("Lower bounds",
            ["ℓst = $(lkt1/100), ℓjt = $(lmt1/100)","ℓst = $(lkt2/100), ℓjt = $(lmt2/100)"], 
            [color_y1, color_y2]))
#     push!(p, Guide.colorkey(title="Lower bounds", labels=["ℓkt = $lkt1, ℓmt = $lmt1", "ℓkt = $lkt2, ℓmt = $lmt2"]))
    push!(p, Guide.xlabel("θt"))
    push!(p, Guide.ylabel("Expected Payoff"))
    push!(p, Guide.title("Expected payoff of function $f"))
    
    ticksx = collect(cut/100:0.05:hhigh)
    push!(p, Guide.xticks(ticks=ticksx))

    

end





s_propose

In [ ]:
#variando lkt

fig1 = graf1(1, 50, 90, 50, 50, s_propose)
draw(PNG("fig1.png", dpi=400), fig1)

graf1(1, 50, 90, 50, 50, s_propose)

In [ ]:
#variando lmt
fig2 = graf1(1, 50, 50, 50, 90, s_propose)
draw(PNG("fig2.png", dpi=400), fig2)


graf1(1, 50, 50, 50, 90, s_propose)

s_respond

In [ ]:
#variando lkt

fig3 = graf1(1, 30, 70, 80, 80, s_respond);
draw(PNG("fig3.png", dpi=400), fig3)

graf1(1, 30, 70, 80, 80, s_respond)

In [ ]:
#variando lmt


fig4 = graf1(1, 50, 50, 50, 90, s_respond);
draw(PNG("fig4.png", dpi=400), fig4)

graf1(1, 50, 50, 50, 90, s_respond)

In [ ]:
function graf_func_value(t, lkt, lmt)
    
    
    set_default_plot_size(16cm, 10cm)
    
    #now the cut is just lkt
    X=hvals[lkt:end]
    
    y1=s_W[t, lkt:end, lkt, lmt]
    y2=j_W[t, lkt:end, lkt, lmt]
    
    
    #plotting
    
    color_y1 = "#1f77b4"
    color_y2 = "#ff7f0e"
    
    
#     possible line_styles=[:solid :dash :dot :dashdot :dashdotdot]
    p = plot()
    
    push!(p, layer(x=X, y=y1, Geom.line,
            Theme(default_color=color(color_y1),
                line_style=[:dash],
                line_width=0.4mm)))
    
    
    push!(p, layer(x=X, y=y2, Geom.line, 
            Theme(default_color=color(color_y2),
                line_style=[:dot],
                line_width=0.4mm)))
    
    


    push!(p, Guide.manual_color_key("Creditor",
            ["S","J"], 
            [color_y1, color_y2]))
    push!(p, Guide.xlabel("θt"))
    push!(p, Guide.ylabel("Expected Payoff"))
    push!(p, Guide.title("Continuation values at t=$(t-1), given ℓst=$(lkt/100) and ℓst=$(lmt/100)"))
    
    ticksx = collect(lkt/100:0.05:hhigh)
    push!(p, Guide.xticks(ticks=ticksx, orientation=:vertical))
    
    
    

    
  
    
    
end

fig5 = graf_func_value(1,1,1);
draw(PNG("fig5.png", dpi=400), fig5)
graf_func_value(1,1,1)

Proposals

In [ ]:

function graf_proposal(f, t)
    
    set_default_plot_size(16cm, 10cm)
    
    #creates a dataframe to store the values
    df = DataFrame()
    df.hk = repeat(1:grid, inner=grid)
    df.lm = repeat(1:grid, outer=grid)
    
    Policy= zeros( size(df,1) )
    
    for i in 1:size(df,1)
#         Policy[i] = f(t, df.hk[i], 1, df.lm[i])[2]
        Policy[i] = f(t, df.hk[i], df.hk[i], df.lm[i])[2]
    end
    
    df.Policy = Policy
    
#     labeldict = Dict(1=>"liq", 2=>"wait", 3=>"reorg")
    
    
#     p = plot(df, x=:hk, y=:lm, color=:Policy, Geom.point, 
#         Scale.color_discrete(levels=[1, 2, 3]))  
    
    #mudanças de cores, ficou meio jamaica
#     p = plot(df, x=:hk, y=:lm, color=:Policy, Geom.point,
#         Scale.color_discrete_manual("red","yellow", "green", levels=[1.0, 2.0, 3.0]))
    
    
    #para ficar comparável ao artigo
    p = plot(df, x=:hk, y=:lm, color=:Policy, Geom.point, alpha=[0.7],
        Scale.color_discrete_manual("black","red", "lightskyblue3", levels=[1.0, 2.0, 3.0]))
    
    push!(p, Guide.xlabel("θt"))
    push!(p, Guide.ylabel("Opponent's lower bound"))
    push!(p, Guide.title("Optimal policy, $f, at t=$(t-1)"))
    push!(p, Guide.colorkey(title="Optimal Policy", labels=["Liquidate", "Wait", "Reorganize"]))
    
                
                
end
            
#ficou muito pesado
            

In [ ]:
graf_proposal(s_propose, 1)

In [ ]:
graf_proposal(s_propose, 3)

In [ ]:
graf_proposal(j_propose,1)

In [ ]:
graf_proposal(j_propose,3)

Analisando os valores estranhos no gráfico j_propose em t==3

In [ ]:
# show(stdout, "text/plain", Pjt_array[3, 40, 30:50, :])


In [ ]:
# U[3, 10]

In [ ]:
# s_W[4, 10, 10, 10]

In [ ]:
function dot_U(t, hkt, grid, U, pkt, pmf)
    
    #calculates the dot product between U and pmf
    payoff=0.0
    @inbounds for i in hkt:grid
        payoff+= (U[t, i] - pkt) * pmf[hkt, i]
    end
    return payoff
end

In [ ]:
# show(stdout, "text/plain",U[3, 40] .*share)